A little bit of segue on the previous notebook, I've recreated another jupyter to explore more on matrix factorization for collaborative filtering without the use of TF (did gone overboard with previous notebook)

Goal in mind being using numpy and scipy for mf svd or nmf with a little bit more control on matrix factorization for different federated clients as I'll explore later.

In [100]:
from src.data_examples.ex2_data_loader import ExampleDataLoader

from enum import Enum, auto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import svds as scipy_svds

pd.options.mode.chained_assignment = None

In [101]:
RAND_SEED = 1337

In [102]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)


c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.
  return func(*args, **kwargs)


In [103]:
# to trucate any other features outside of matrix factorization
# select only user_id, movie_id, and rating for our matrix

features_X = ['user_id', 'movie_id']
feature_Y = 'rating'

df_main = data_loader.df[features_X + [feature_Y]]


for feature in features_X:
  df_main[feature] = df_main[feature].astype('int32')

df_main

,user_id,movie_id,rating
0,195,241,0.2
1,304,241,0.6
2,5,241,0.4
3,233,241,0.4
4,62,241,0.2
...,...,...,...
99995,862,1678,0.2
99996,862,1677,-0.2
99997,862,1679,0.0
99998,895,1680,0.2


In [104]:
def df_to_np_sparse_matrix(df, U_c, V_c, val_c):
  # pivot the data with movie_id as column to create sparse mxn matrix 
  df = df[[U_c, V_c, val_c]]
  df_pivot = df.pivot(index=U_c, columns=V_c, values=val_c).fillna(0)
  return df_pivot.values

def sparse_matrix_normalize(mat_sparse):
  # normalize matrix from means of axis 1
  mat_mean = np.mean(mat_sparse, axis=1)
  return (mat_sparse - mat_mean.reshape(-1, 1)), mat_mean

def eval_svd_factorization(df, U_c, V_c, val_c, latent_factor=25):
  mat_sparse = df_to_np_sparse_matrix(df_main, 'user_id', 'movie_id', 'rating')
  mat_sparse_norm, mat_sparse_mean = sparse_matrix_normalize(mat_sparse)

  U, sigma, Vt = scipy_svds(mat_sparse, k=latent_factor)
  sigma = np.diag(sigma)

  mat_prediction = np.dot(np.dot(U, sigma), Vt) + mat_sparse_mean.reshape(-1, 1)

  return mat_prediction, (U, sigma, Vt)


In [105]:
mat_pred, (U, sigma, Vt) = eval_svd_factorization(df_main, 'user_id', 'movie_id', 'rating', latent_factor=25)

df_pred = pd.DataFrame(mat_pred) 
df_pred # which will directly encodes user_id as index axis 1 and movie_id as columns

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.591173,0.218883,0.216366,0.320632,0.074675,0.128291,0.496458,0.195799,0.381042,0.227432,...,0.050663,0.053200,0.050407,0.052081,0.052497,0.052149,0.052012,0.052081,0.054034,0.059397
1,0.239390,-0.008896,-0.025523,0.045449,0.015504,0.047360,0.121989,0.106183,0.207451,0.064969,...,0.012884,0.010631,0.013449,0.012604,0.011902,0.011734,0.013475,0.012604,0.013509,0.010338
2,0.042691,0.010405,0.026525,-0.016571,-0.013632,0.002582,-0.035287,-0.015781,-0.047577,0.009940,...,0.004925,0.003757,0.006038,0.005113,0.004857,0.003596,0.006630,0.005113,0.005093,0.005625
3,0.064871,-0.016956,0.025028,-0.014872,0.017362,0.010358,0.062990,-0.010724,0.058845,-0.002500,...,0.006617,0.005249,0.006122,0.006659,0.007138,0.005123,0.008194,0.006659,0.006491,0.006808
4,0.395072,0.085839,0.024492,0.187126,0.004489,0.000817,0.323427,0.142048,-0.114113,0.044969,...,0.017919,0.018186,0.017251,0.018193,0.016831,0.018249,0.018136,0.018193,0.019005,0.017480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.339086,0.015353,0.011655,0.019144,0.056034,0.024695,0.145706,0.102731,0.168634,0.023343,...,0.014449,0.012582,0.014152,0.013199,0.013107,0.014511,0.011886,0.013199,0.013769,0.011267
939,0.044876,0.006287,-0.008784,0.220981,0.060224,0.013593,0.296255,0.238947,0.194064,0.010604,...,0.018983,0.023172,0.018715,0.018549,0.019140,0.017507,0.019591,0.018549,0.018872,0.017431
940,0.262788,0.015168,0.027576,0.006428,-0.001998,0.005935,0.189606,0.089612,0.125642,0.024956,...,0.004565,0.003910,0.005605,0.005351,0.006312,0.005493,0.005208,0.005351,0.006143,0.005221
941,0.135355,0.030077,-0.000674,0.020363,0.003278,0.003302,0.000714,0.173069,0.007655,0.024929,...,0.022885,0.023883,0.022420,0.021284,0.021475,0.019655,0.022914,0.021284,0.022923,0.020821


In [106]:
tgt_user_id = 420
n_movies = 15

pred_movie_ids = df_pred.iloc[tgt_user_id].sort_values(ascending=False)[:n_movies].index


__fmt_movie = lambda movie_id: f'movie_id: {movie_id}'.ljust(17) + data_loader.df_movies.loc[movie_id]['title']

pred_movie_ids = df_main[df_main['user_id'] == tgt_user_id]['movie_id'] [:n_movies]
print('recommendtion')
for movie_id in pred_movie_ids:
  print(__fmt_movie(movie_id))



recommendtion
movie_id: 237    Raising Arizona (1987)
movie_id: 172    Princess Bride, The (1987)
movie_id: 268    Full Monty, The (1997)
movie_id: 301    L.A. Confidential (1997)
movie_id: 332    Game, The (1997)
movie_id: 116    Rock, The (1996)
movie_id: 99     Fargo (1996)
movie_id: 78     Fugitive, The (1993)
movie_id: 97     Silence of the Lambs, The (1991)
movie_id: 55     Pulp Fiction (1994)
movie_id: 11     Usual Suspects, The (1995)
movie_id: 330    Edge, The (1997)
movie_id: 95     Terminator 2: Judgment Day (1991)
movie_id: 175    Aliens (1986)
movie_id: 207    Young Frankenstein (1974)


## Federated Learning with Clients (or Silos) Manager

In [127]:
class DBKind(Enum):
  shared = auto()
  private = auto()

In [128]:
# crude emulation server-clients with ClientManager

class Client:
  def __init__(self, _uuid=None):
    self._task_pool = []
    self._private_dbs = {}
    self._shared_dbs = {}
    self._mem = {}
    if _uuid is None:
      self._uuid = id(self) # fallbacks to basically the address on memory of client object for the sake of simplicity
    else:
      self._uuid = _uuid

  def get_uuid(self):
    return self._uuid

  def add_task(self, fn):
    self._task_pool.append(fn)

  def run_tasks(self):
    for task in self._task_pool:
      task(self)

  def clear_tasks(self):
    self._task_pool.clear()

  def clear_mem(self):
    self._mem.clear()

  def add_db(self, kind: DBKind, name, db):
    if kind == DBKind.shared:
      self._shared_dbs[name] = db
    elif kind == DBKind.private:
      self._private_dbs[name] = db

  def get_db(self, kind: DBKind, name):
    if kind == DBKind.shared:
      return self._shared_dbs[name]
    elif kind == DBKind.private:
      return self._private_dbs[name]


class ClientManager:
  def __init__(self):
    self._client_bucket = []
    self._client_uuids = []

  def new_client(self, _uuid=None):
    client = Client(_uuid=_uuid)
    self._client_bucket.append(client)
    self._client_uuids.append(client.get_uuid())
    if _uuid is None:
      client._uuid = id(client)
    else:
      client._uuid = _uuid

    return client

  def get_client_uuids(self):
    return self._client_uuids

  def get_client_by_uuid(self, _uuid):
    if _uuid not in self._client_uuids:
      return None
    return self._client_bucket[self._client_uuids.index(_uuid)]

  def force_run_tasks(self):
    for client in self._client_bucket:
      client.run_tasks()

  def force_clear_tasks(self):
    for client in self._client_bucket:
      client.clear_tasks()
    
  def force_clear_mems(self):
    for client in self._client_bucket:
      client.clear_mem()

# example
manager = ClientManager()
for _ in range(4):
  client = manager.new_client()
  client.add_task(lambda this: print(f'running task on client: print with uuid {this.get_uuid()}'))

print('client uuids:', manager.get_client_uuids())
manager.force_run_tasks()

client uuids: [2418137383936, 2418381671248, 2418381678688, 2418381671968]
running task on client: print with uuid 2418137383936
running task on client: print with uuid 2418381671248
running task on client: print with uuid 2418381678688
running task on client: print with uuid 2418381671968


In [129]:
unique_zip_codes = data_loader.df_users['zip_code'].unique()

# change to len(unique_zip_codes) for individual zip codes to be assigned as individual client
n_zip_code_clusters = 10

unique_zip_codes_clusters = np.array_split(unique_zip_codes, n_zip_code_clusters)

In [130]:
unique_zip_codes = data_loader.df_users['zip_code'].unique()
user_ratings_headers = ['zip_code', 'user_id', 'movie_id', 'rating']

manager = ClientManager()

for zip_codes in unique_zip_codes_clusters:
  uuid_name = '-'.join(zip_codes[:3]) + '...'
  client_ref = manager.new_client(_uuid=uuid_name)
  client_ref.add_db(kind=DBKind.shared, name='df_movies', db=data_loader.df_movies)

  zip_code_mask = data_loader.df['zip_code'].apply(lambda x: x in zip_codes)
  private_df = data_loader.df[zip_code_mask]
  private_df = private_df[user_ratings_headers].copy()
  client_ref.add_db(kind=DBKind.private, name='df_user_ratings', db=private_df)


### adding svd fit tasks to each clients

In [139]:
manager.force_clear_tasks()

In [140]:
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  client_ref.add_task(
    lambda client: print(f'running task on client {client.get_uuid()} with num of users: {client.get_db(DBKind.private, "df_user_ratings")["user_id"].unique().shape[0]}')
  )

  client_ref._mem['latent_factor'] = 50

# try to run tasks on client
manager.force_run_tasks()


running task on client 85711-94043-32067... with num of users: 129
running task on client 20003-46005-89503... with num of users: 104
running task on client 53705-30067-78750... with num of users: 97
running task on client 39042-77005-77801... with num of users: 92
running task on client 76059-45660-61455... with num of users: 92
running task on client 85282-33308-53202... with num of users: 94
running task on client 02324-05464-80302... with num of users: 89
running task on client 48043-74101-01940... with num of users: 86
running task on client 98405-85719-98072... with num of users: 81
running task on client 90814-47401-93055... with num of users: 79


In [141]:
def task_eval_svd_factorization(client):
  print('begin factorization ...' end='')
  df = client.get_db(DBKind.private, 'df_user_ratings')
  latent_factor = client._mem['latent_factor']
  mat_pred, (U, sigma, Vt) = eval_svd_factorization(df, 'user_id', 'movie_id', 'rating', latent_factor=latent_factor)
  client._mem['U'] = U
  client._mem['sigma'] = sigma
  client._mem['Vt'] = Vt
  print('finished factorization\n')

for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  client_ref.add_task(task_eval_svd_factorization)



running task on client 85711-94043-32067... with num of users: 129
begin factorization


finished factorization

running task on client 20003-46005-89503... with num of users: 104
begin factorization
finished factorization

running task on client 53705-30067-78750... with num of users: 97
begin factorization
finished factorization

running task on client 39042-77005-77801... with num of users: 92
begin factorization
finished factorization

running task on client 76059-45660-61455... with num of users: 92
begin factorization
finished factorization

running task on client 85282-33308-53202... with num of users: 94
begin factorization
finished factorization

running task on client 02324-05464-80302... with num of users: 89
begin factorization
finished factorization

running task on client 48043-74101-01940... with num of users: 86
begin factorization
finished factorization

running task on client 98405-85719-98072... with num of users: 81
begin factorization
finished factorization

running task on client 90814-47401-93055... with num of users: 79
begin factorization
finished 

In [ ]:
# run all factorization tasks
manager.force_run_tasks()

In [144]:
# fetch all client saved memory weakrefs, as variables in this case

client_mems = {}
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)
  client_mems[client_ref.get_uuid()] = {}

  for k in ['U', 'sigma', 'Vt']:
    client_mems[client_ref.get_uuid()][k] = client_ref._mem[k]


{'85711-94043-32067...': {'U': array([[-0.04183966,  0.11164012,  0.07938858, ...,  0.01345962,
          -0.00365122,  0.06902209],
         [ 0.01128885,  0.00470685,  0.02031772, ...,  0.07331988,
           0.02282591,  0.01457891],
         [-0.00418777, -0.00858666, -0.00913058, ...,  0.01624782,
           0.00719375,  0.00345885],
         ...,
         [-0.02495138,  0.0248624 , -0.01538404, ...,  0.019831  ,
           0.02186059,  0.00789713],
         [-0.04321725,  0.04103867,  0.02273993, ..., -0.00245595,
          -0.00952792,  0.02871656],
         [ 0.1017458 ,  0.03285124, -0.00400568, ..., -0.02756564,
           0.0344398 ,  0.0404784 ]]),
  'sigma': array([[ 6.46480944,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  6.4918796 ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  6.5338638 , ...,  0.        ,
           0.        ,  0.        ],
         .